# Train VoiceD/VoicelesS discriminator
Using the latest NN phone alignment, grnerate D/S training data and train a NN on it.
Silence is also considered as voiceless.

In [ ]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl
###%run ../acmodel/matrix.py
%run ../acmodel/praat_ifc.py
%run ../acmodel/hmm_acmodel.py

device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f"Using {device} device")

%run ../acmodel/nn_acmodel.py

In [ ]:
device

## Get training data
We previously aligned Czech CommonVoice train set using an ultra-prinmitive HMM/GMM and then NNs. Let's replace every non-silent phone label by either 'c' or 'v' (so ve have 3 labels: "cv|").

In [ ]:
infile = "mega6_training_0028.tsv"
df = pd.read_csv(infile, sep="\t", keep_default_na=False)
hmms = get_training_hmms(infile, derivatives=3)
b_log_corr = b_log_corrections(infile) # get b() corrections based on frequency

In [ ]:
all_mfcc, all_targets, b_set = collect_training_material(hmms)

#out_size = len(b_set)
in_size = hmms[0].mfcc.size(1)

" ".join(b_set), in_size

### Map phone labels voiceD/voicelesS

In [ ]:
lab = {}
for phone in b_set:
    lab[phone] = 's' # default to be overwriten below
for voiced in 'AEOaeouyáéóúý'+'GNZbdghjlmnrvzďňřŽž':
    lab[voiced] = 'd'

In [ ]:
all_targets[:200]

In [ ]:
ds_targets = "".join(lab[p] for p in all_targets)

In [ ]:
ds_targets[:200]

In [ ]:
ds_set = sorted({*ds_targets})
ds_set

In [ ]:
out_size = len(ds_set)

## Setup PyTorch training tools

In [ ]:
in_size, out_size

In [ ]:
model = NeuralNetwork(in_size, out_size).to(device)
print(model)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
training_data = SpeechDataset(all_mfcc, ds_targets, ds_set) # initial alignment
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

for mega_epoch in range(100):
    print(f"============ Training DS Epoch {mega_epoch} =============")
    train_n_epochs(train_dataloader, optimizer, model, criterion, 5)
    filename_base = f"ds_training_{'%04d' % mega_epoch}"
    torch.save(model.state_dict(), filename_base+".pth")
